In [3]:
import pandas
import re

data = pandas.read_excel("AMAM eMuseum data export 9.26.24.xlsx")
data["Dated"]

0             19th century
1                     1856
2                  1855–56
3                 ca. 1853
4       early 20th century
               ...        
2144                  2022
2145              ca. 1883
2146                  1983
2147                  2024
2148                  1991
Name: Dated, Length: 2149, dtype: object

In [23]:
def parse(entry):
    m = re.fullmatch(r"(\d+)(?:st|nd|rd|th) century", entry)
    if m:
        century = int(m.group(1)) * 100
        return century - 99, century, False
    m = re.fullmatch(r"(\d+)", entry)
    if m:
        year = int(m.group(1))
        return year, year, False
    m = re.fullmatch(r"(\d{2,})(?:-|—|–)(\d{2})", entry)
    if m:
        start_year = int(m.group(1))
        end_year = start_year - (start_year % 100) + int(m.group(2))
        return start_year, end_year, False
    m = re.fullmatch(r"(?:ca. |circa )(\d+)", entry)
    if m:
        year = int(m.group(1))
        return year, year, True
    m = re.fullmatch(r"early (\d{1,2})(?:st|nd|rd|th) century(?: CE)?", entry)
    if m:
        start_year = int(m.group(1)) * 100 - 99
        end_year = start_year + 33
        return start_year, end_year, True
    m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)(?:-|—|–)(\d{1,2})(?:st|nd|rd|th) century", entry)
    if m:
        start_year = int(m.group(1)) * 100 - 99
        end_year = start_year + 99
        return start_year, end_year, False
    m = re.fullmatch(r"late (\d{1,2})(?:st|nd|rd|th) century", entry)
    if m:
        start_year = int(m.group(1)) * 100 - 33
        end_year = start_year + 33
        return start_year, end_year, True
    m = re.fullmatch(r"(\d{3}0)s", entry)
    if m:
        start_year = int(m.group(1))
        end_year = start_year + 9
        return start_year, end_year, False

    raise ValueError(f"Failed to parse {entry}")


data[["Start Date", "End Date", "Approximate"]] = data["Dated"].map(parse)

ValueError: Failed to parse second half 17th–early 18th century